In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64

from collections import OrderedDict


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

In [2]:
###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

# CRUD object 
# user must pass username and password for authentication to the AAC animals collection
username = "aacuser"
password = "Admin9821"
database_name = "AAC"
collection_name = "animals"
port_number = 32614
shelter = AnimalShelter(database_name, port_number, username, password, collection_name)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))

In [3]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
                               
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H5('Sergio H. Passos'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image',
             style={
                    'width' : '5%',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0}),
    html.Div(      
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount_rescue'},
                {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster_rescue'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        ),
        style={
            'padding-top': '1%'
        }
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable =True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10, 
    ),
    html.Br(),
    html.Hr(),
# this sets up the layout so that the map and pie chart are place horizontally to each other
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
        html.Div(
            id='graph-id',
            className='col s12 m6',

            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



# filter callback update
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    ### updates the data table, and bar chart
    
    # water_resecue filter toggled
    if filter_type == 'water_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"
                             ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))
    # mount_rescue filter toggled
    elif filter_type == 'mount_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog", 
                              "Siberian Husky", "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))

    # disaster_rescue filter toggled
    elif filter_type == 'disaster_rescue':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", 
                              "Bloodhound","Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))

    # defaults to reset being selected
    else:
        # resets the query to no filter and gets all the data
        df = pd.DataFrame.from_records(shelter.read({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')

    # returns the output data for the datatable
    return (data,columns)

# change background color of selected field
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#f4a261'
    } for i in selected_columns]


# bar chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(data):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(data)
    
    # organize the data by breed
    dff = dff.breed.value_counts() # returns a descending order Series
    dff.name = "breed" # use the breed type as the name
    dff = dff.to_frame() # turns the Series object into a DataFran=me
    dff.reset_index(inplace=True)
    
    return [
        dcc.Graph(            
            figure = px.pie(dff, values='breed', names='index', title='Data Visualized', hole=0.25)
        )    
    ]

# Map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'derived_viewport_selected_rows')])
def update_map(viewData, row_ids):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets first num in row_ids array to the last selected row ID
    row_ids[0] = row_ids[len(row_ids) - 1]
    return [
        #
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[row_ids[0],13],dff.iloc[row_ids[0],14]], children=[
                # parameter num 4 gives the breed of animal
                dl.Tooltip(dff.iloc[row_ids[0],4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    # parameter 9 gives animal's name
                    html.P(dff.iloc[row_ids[0],9])
                ])
            ])
        ])
    ]


app